In [1]:
import pathlib
import os
import time
import zipfile

from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SparkSession
from tqdm.notebook import tqdm
import xmltodict

In [11]:
with zipfile.ZipFile("rsmp/data-01102019-structure-08012016.zip") as f:
    items = f.infolist()
    for item in items[:len(items) // 10]:
        f.extract(item, path = "rsmp/extract")

In [21]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.12.0 pyspark-shell'
conf = SparkConf()
conf.setMaster("local[*]").setAppName("XML Processing Performance Tester")

In [22]:
sc = SparkContext(conf=conf)
session = SparkSession(sc)

23/07/29 13:35:47 WARN Utils: Your hostname, asus-notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.186 instead (on interface wlp3s0)
23/07/29 13:35:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pavel/.venv/hse/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pavel/.ivy2/cache
The jars for the packages stored in: /home/pavel/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e6e04a8-0457-465f-9f90-d91f7a72e50a;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 514ms :: artifacts dl 26ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.12.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| se

In [2]:
samples_path = pathlib.Path("rsmp/extract")

# pandas

In [38]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f)

0it [00:00, ?it/s]

CPU times: user 1min 44s, sys: 1.6 s, total: 1min 46s
Wall time: 1min 52s


# pandas + spark

In [42]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f)
    df.to_csv("rsmp/joint4.csv", mode="a")

0it [00:00, ?it/s]

CPU times: user 1min 32s, sys: 2.12 s, total: 1min 34s
Wall time: 1min 37s


In [31]:
%%time
df = (
    session
    .read
    .csv("rsmp/joint.csv", header=True)
)

CPU times: user 4.13 ms, sys: 0 ns, total: 4.13 ms
Wall time: 563 ms


In [32]:
df.show(5)

+---+------+--------------------+----------+----------+---------+---------+----------+---------+------+-------+--------+--------+
|_c0|ФИООтв|               ИдДок|  ДатаСост|ДатаВклМСП|ВидСубМСП|КатСубМСП|ПризНовМСП|ОргВклМСП|СведМН|СвОКВЭД|СвЛиценз|ИПВклМСП|
+---+------+--------------------+----------+----------+---------+---------+----------+---------+------+-------+--------+--------+
|  0|  null|                null|      null|      null|     null|     null|      null|     null|  null|   null|    null|    null|
|  1|  null|6c342c1a-8c49-caf...|10.01.2019|10.08.2018|      1.0|      1.0|       2.0|     null|  null|   null|    null|    null|
|  2|  null|a363ad3a-2b65-522...|10.01.2019|10.03.2017|      1.0|      1.0|       2.0|     null|  null|   null|    null|    null|
|  3|  null|de4ad63c-db88-bc7...|10.01.2019|10.09.2016|      1.0|      1.0|       2.0|     null|  null|   null|    null|    null|
|  4|  null|c9e64a12-0882-bfb...|10.01.2019|01.08.2016|      1.0|      1.0|       2.0|    

23/07/29 15:40:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ФИООтв, ИдДок, ДатаСост, ДатаВклМСП, ВидСубМСП, КатСубМСП, ПризНовМСП, ОргВклМСП, СведМН, СвОКВЭД, СвЛиценз, ИПВклМСП
 Schema: _c0, ФИООтв, ИдДок, ДатаСост, ДатаВклМСП, ВидСубМСП, КатСубМСП, ПризНовМСП, ОргВклМСП, СведМН, СвОКВЭД, СвЛиценз, ИПВклМСП
Expected: _c0 but found: 
CSV file: file:///media/data/Павел/Документы/Учёба/Аспирантура/tax-service-opendata/rsmp/joint.csv


# pandas xslt

In [51]:
df = pd.read_xml("rsmp/VO_RRMSPSV_0000_9965_20170410_0a1a5aa4-6a6d-4f93-91b6-ead7ba8f83c3.xml", stylesheet="rsmp.xsl")

In [52]:
df

,kind,category,in_date,ind_tin,first_name,last_name,patronymic,org_name,org_short_name,org_tin,region_code,region_name,region_type,city_name,city_type,settlement_name,settlement_type,activity_code_main,activity_codes_additional,total
0,NaN,NaN,None,NaN,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,NaN,None,NaN
1,2.0,1.0,01.08.2016,7.444103e+11,ДМИТРИЙ,НИКИТИН,НИКОЛАЕВИЧ,NaN,NaN,NaN,74.0,ЧЕЛЯБИНСКАЯ,ОБЛАСТЬ,МАГНИТОГОРСК,ГОРОД,NaN,NaN,69.1,"82.99, 66.19.4, 70.22, 68.10.21, 45.11.3, 45.1...",900.0
2,2.0,1.0,01.08.2016,7.445000e+11,СЕРГЕЙ,БОРОВЛЕВ,ГАВРИЛОВИЧ,NaN,NaN,NaN,74.0,ЧЕЛЯБИНСКАЯ,ОБЛАСТЬ,МАГНИТОГОРСК,ГОРОД,NaN,NaN,69.1,"43.32, 45.11, 43.12, 45.20, 45.31, 43.33, 41.2...",900.0


In [3]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f, stylesheet="rsmp.xsl")
    df.to_csv("joint.csv", mode="a")

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [54]:
with open("rsmp/joint3.csv") as f:
    print(f.read(500))

,kind,category,in_date,ind_tin,first_name,last_name,patronymic,org_name,org_short_name,org_tin,region_code,region_name,region_type,city_name,city_type,settlement_name,settlement_type,activity_code_main,activity_codes_additional
0,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,10.02.2017,,,,,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Г.Е.Н.И.Я.""","ООО ""Г.Е.Н.И.Я.""",8602274490.0,86.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА,АВТОНОМНЫЙ ОКРУГ,СУРГУТ,ГОРОД,,,69.1,"69.20.3,"
2,1.0,1.0,10.10.2016,,,,,"ОБЩЕСТВО С ОГР


In [57]:
pd.read_csv("rsmp/joint3.csv", index_col=0)

,kind,category,in_date,ind_tin,first_name,last_name,patronymic,org_name,org_short_name,org_tin,region_code,region_name,region_type,city_name,city_type,settlement_name,settlement_type,activity_code_main,activity_codes_additional,total
0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,1.0,1.0,10.02.2017,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Г.Е....","ООО ""Г.Е.Н.И.Я.""",8602274490.0,86.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА,АВТОНОМНЫЙ ОКРУГ,СУРГУТ,ГОРОД,NaN,NaN,69.1,"69.20.3,",900.0
2.0,1.0,1.0,10.10.2016,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮРИД...","ООО ""ЮРКОНСАЛТИНГ""",8602272084.0,86.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА,АВТОНОМНЫЙ ОКРУГ,СУРГУТ,ГОРОД,NaN,NaN,69.1,"73.20, 73.11, 68.10, 81.21, 68.31, 69.20, 78.1...",900.0
3.0,1.0,1.0,01.08.2016,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦЕНТ...","ООО ""ЦЕНТР ПРАВОВОЙ ПОМОЩИ""",8602270175.0,86.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА,АВТОНОМНЫЙ ОКРУГ,СУРГУТ,ГОРОД,NaN,NaN,69.1,"68.31.52, 68.20.1, 68.31.12, 68.31.32, 69.20.3...",900.0
4.0,1.0,1.0,10.12.2016,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕГИ...","ООО ""РИ""",8602273546.0,86.0,ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА,АВТОНОМНЫЙ ОКРУГ,СУРГУТ,ГОРОД,NaN,NaN,69.1,"68.10, 45.20.2, 82.99, 47.99, 49.41.1, 52.10, ...",900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.0,2.0,1.0,01.08.2016,234912496518.0,АННА,ТАРАНЕЦ,СЕРГЕЕВНА,NaN,NaN,NaN,23.0,Краснодарский,Край,NaN,NaN,Анастасиевская,Станица,69.1,"68.10, 47.19.2, 68.31, 68.10.11, 47.19.1, 68.1...",900.0
4.0,2.0,1.0,10.05.2017,234910174508.0,АЛЕКСЕЙ,ЛИТВИНЕНКО,ДМИТРИЕВИЧ,NaN,NaN,NaN,23.0,Краснодарский,Край,Краснодар,Город,NaN,NaN,69.1,"74.10, 43.91, 82.19, 66.21, 14.13, 43.99, 43.3...",900.0
5.0,2.0,1.0,01.08.2016,234912434102.0,АЛЕКСАНДР,МЕЛИШЕВ,ИВАНОВИЧ,NaN,NaN,NaN,23.0,Краснодарский,Край,Краснодар,Город,NaN,NaN,69.1,"43.31, 43.33, 43.34.2, 41.20, 73.20, 43.34.1, ...",900.0
6.0,2.0,1.0,10.10.2017,234912382408.0,НАТАЛЬЯ,АКСЕНОВА,АЛЕКСАНДРОВНА,NaN,NaN,NaN,23.0,КРАСНОДАРСКИЙ,КРАЙ,СЛАВЯНСК-НА-КУБАНИ,ГОРОД,NaN,NaN,69.1,"45.20.2, 71.20.6, 65.12.3, 63.11, 33.17, 74.90...",900.0


In [59]:
df = pd.read_xml("revexp/VO_OTKRDAN_5_9965_9965_20230626_0a24217e-8d61-4d2a-8487-ffa1fb686028.xml", stylesheet="revexp.xsl")
df

,org_name,org_tin,revenue,expediture
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СПВ ...",4205298674,0.0,0.0
1,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮНИОР""",7703821980,0.0,0.0
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРИАДА""",9102044388,1612000.0,1123000.0
3,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НУТР...",6321368816,0.0,12000.0
4,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НОМЕР""",1831169720,5415000.0,6396000.0
...,...,...,...,...
895,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АРГО...",9201012098,9034000.0,2047000.0
896,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СИВА...",9106002300,11342000.0,9109000.0
897,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФОРК""",278217060,10021000.0,9950000.0
898,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛИГА...",6234137940,423000.0,392000.0


In [ ]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f, stylesheet="rsmp.xsl")
    df.to_csv("rsmp/joint_revexp.csv", mode="a")

In [61]:
df = pd.read_xml("sshr/VO_OTKRDAN_3_9965_9965_20230625_0a7bc178-6202-428a-9546-80b9cc00f7dc.xml", stylesheet="sshr.xsl")
df

,org_name,org_tin,employees_count,date
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МОЗЕР""",6671057542,1,31.12.2022
1,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛЕКОПТ""",9718030230,91,31.12.2022
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БРАСКО""",7720355560,0,31.12.2022
3,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ШТОЛ...",7724384788,6,31.12.2022
4,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГИДР...",5029214780,0,31.12.2022
...,...,...,...,...
895,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СЕРВ...",5029214808,2,31.12.2022
896,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЕВРО...",1659175780,2,31.12.2022
897,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛИТТ...",245953339,2,31.12.2022
898,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОМП...",6321417894,1,31.12.2022


# pandas xslt parallel

In [2]:
from concurrent import futures
import multiprocessing

In [15]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f, stylesheet="rsmp.xsl")
    df.to_csv("joint_single-thread.csv", mode="a")

0it [00:00, ?it/s]

CPU times: user 1min 13s, sys: 5.21 s, total: 1min 18s
Wall time: 1min 22s


In [5]:
class Archive:
    def __init__(self, path):
        self._archive = zipfile.ZipFile(path)
        self._xml_list = [f for f in self._archive.infolist() if "xml" in f.filename]

    def __del__(self):
        self._archive.close()

    def __len__(self):
        return len(self._xml_list)

    def __next__(self):
        f = next(iter(self._xml_list))
        return self._archive.read(f).decode()

    def __iter__(self):
        for f in iter(self._xml_list):
            yield self._archive.read(f).decode()

    def __getitem__(self, index):
        f = self._xml_list[index]
        return self._archive.read(f).decode()
        

In [7]:
a = Archive("rsmp/data/data-01102019-structure-08012016.zip")

In [40]:
for i in a:
    print(i[:10])
    break

b'<?xml vers'


In [42]:
%%time
def func(item):
    with zipfile.ZipFile("rsmp/data/data-01102019-structure-08012016.zip") as archive:
        xml = archive.open(item)
        
    return pd.read_xml(xml, stylesheet="rsmp.xsl")

with zipfile.ZipFile("rsmp/data/data-01102019-structure-08012016.zip") as archive:
    items = list(archive.infolist())
    print(len(items))
    
with multiprocessing.Pool(processes=4) as pool, open("joint_b.csv", "a") as f:
    for df in pool.imap(func, items, chunksize=10):
        df.to_csv(f, mode="a")

6712
CPU times: user 13.9 s, sys: 628 ms, total: 14.6 s
Wall time: 10min 20s


In [6]:
%%time
def func(item):        
    return pd.read_xml(item, stylesheet="rsmp.xsl")

a = Archive("rsmp/data/data-01102019-structure-08012016.zip")

with multiprocessing.Pool(4) as pool, open("joint_a3.csv", "a") as f:
    for df in pool.imap(func, a, chunksize=8):
        df.to_csv(f, mode="a")

CPU times: user 4min 11s, sys: 13.2 s, total: 4min 24s
Wall time: 5min 45s


In [16]:
from multiprocessing.pool import ThreadPool

In [19]:
%%time
def func(item):        
    return pd.read_xml(item, stylesheet="rsmp.xsl")

a = Archive("rsmp/data/data-01102019-structure-08012016.zip")
    
with ThreadPool(processes=4) as pool, open("joint_threads.csv", "a") as f:
    for df in pool.imap(func, a, chunksize=10):
        df.to_csv(f, mode="a")

CPU times: user 15min 21s, sys: 9.84 s, total: 15min 31s
Wall time: 4min 51s


In [25]:
df_st = pd.read_csv("joint_single-thread.csv")
df_mt = pd.read_csv("joint_multithread.csv")
assert df_st.equals(df_mt)

# pandas xslt parquet

In [6]:
%%time
for f in tqdm(samples_path.iterdir()):        
    df = pd.read_xml(f, stylesheet="rsmp.xsl", dtype=str)
    if not os.path.exists("joint.parquet"):
        df.to_parquet("joint.parquet", engine="fastparquet")
    else:
        df.to_parquet("joint.parquet", engine="fastparquet", append=True)

0it [00:00, ?it/s]

ValueError: xpath does not return any nodes or attributes. Be sure to specify in `xpath` the parent nodes of children and attributes to parse. If document uses namespaces denoted with xmlns, be sure to define namespaces and use them in xpath.

In [4]:
! pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.6 MB/s eta 0:00:005.7 MB/s eta 0:00:01m


# lxml + spark: extract - join - load

In [19]:
%%time
with open("rsmp/joint.xml", "w") as out:
    print("<Документы>", file=out)
    for f in tqdm.tqdm(samples_path.iterdir()):        
        for event, element in etree.iterparse(f, tag="Документ"):
            print(etree.tostring(element, encoding="utf8").decode(), file=out)
    print("</Документы>", file=out)

671it [03:16,  3.42it/s]

CPU times: user 1min 49s, sys: 12.9 s, total: 2min 2s
Wall time: 3min 16s


In [24]:
%%time
df = (
    session
    .read
    .format("xml")
    .options(rowTag="Документ")
    .load("rsmp/joint.xml")
)

CPU times: user 220 ms, sys: 35.9 ms, total: 255 ms
Wall time: 12min 23s


# vanilla spark

In [20]:
%%time
df = (
    session
    .read
    .format("xml")
    .options(rowTag="Документ")
    .load(",".join(
            "rsmp/extract/" + f for f in os.listdir("rsmp/extract") if "xml" in f
        )
    )
)

In [21]:
df.printSchema()

root
 |-- _ВидСубМСП: long (nullable = true)
 |-- _ДатаВклМСП: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- _КатСубМСП: long (nullable = true)
 |-- _ПризНовМСП: long (nullable = true)
 |-- ИПВклМСП: struct (nullable = true)
 |    |-- _ИННФЛ: long (nullable = true)
 |    |-- ФИОИП: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _Имя: string (nullable = true)
 |    |    |-- _Отчество: string (nullable = true)
 |    |    |-- _Фамилия: string (nullable = true)
 |-- ОргВклМСП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)
 |    |-- _НаимОргСокр: string (nullable = true)
 |-- СвДог: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _ДатаДог: string (nullable = true)
 |    |    |-- _ИННЮЛ_ЗД: long (n

# lxml: just read

In [14]:
%%time
for f in tqdm.tqdm(os.listdir("rsmp/extract")):
    for event, element in etree.iterparse(
            "rsmp/extract/" + f,
            tag="Документ"):
        ...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 671/671 [01:02<00:00, 10.72it/s]

CPU times: user 54.1 s, sys: 1.42 s, total: 55.5 s
Wall time: 1min 2s


# xmltodict

In [35]:
%%time
for fn in tqdm.tqdm(samples_path.iterdir()):
    with open(fn) as f:
        xmltodict.parse(f.read())

671it [03:24,  3.28it/s]

CPU times: user 3min 13s, sys: 1.93 s, total: 3min 15s
Wall time: 3min 24s


# bs4

In [35]:
with open("rsmp/extract/VO_RRMSPSV_0000_9965_20190110_00019b6b-66db-47a3-9dc1-531f91eed871.xml") as f:
    s = BeautifulSoup(f.read(), features="xml")

In [36]:
%%time
for fn in tqdm.tqdm(samples_path.iterdir()):
    with open(f"rsmp/extract/{fn}") as f:
        s = BeautifulSoup(f.read(), features="xml")

VO_RRMSPSV_0000_9965_20190110_00019b6b-66db-47a3-9dc1-531f91eed871.xml
VO_RRMSPSV_0000_9965_20190110_00097c83-44ac-448c-826b-86b249e212fd.xml
VO_RRMSPSV_0000_9965_20190110_0015eee3-3cf5-408d-97d0-3e03f2a7d29d.xml
VO_RRMSPSV_0000_9965_20190110_001691bf-a015-49b5-b074-b5088f2a4033.xml
VO_RRMSPSV_0000_9965_20190110_0018d482-6717-4293-b57c-ac46e5fe7167.xml
VO_RRMSPSV_0000_9965_20190110_001ce934-a384-4250-ae70-5a3da6ecf159.xml
VO_RRMSPSV_0000_9965_20190110_001cf025-d3ca-4847-af64-8e5fe228c7b0.xml
VO_RRMSPSV_0000_9965_20190110_00230eda-e4a9-417d-9b30-3ddcfb586758.xml
VO_RRMSPSV_0000_9965_20190110_003bb278-bc3f-4cd6-8f0c-8054bba7e4e7.xml
VO_RRMSPSV_0000_9965_20190110_003c43e1-d2b4-436c-8426-8a8ca550ee57.xml
VO_RRMSPSV_0000_9965_20190110_00412def-0e24-4b50-84af-889b97d2b8c1.xml
VO_RRMSPSV_0000_9965_20190110_0049731a-6d6d-4ae6-b4f3-4e56fb221755.xml
VO_RRMSPSV_0000_9965_20190110_00559d02-ded5-48d0-9495-e6b096f26e92.xml
VO_RRMSPSV_0000_9965_20190110_005e0b90-ae4d-4711-a834-187503db715f.xml
VO_RRM

VO_RRMSPSV_0000_9965_20190110_0401e186-b04e-45a9-942c-136a8f2fdf47.xml
VO_RRMSPSV_0000_9965_20190110_0405f025-14eb-4b23-8939-261661718aa0.xml
VO_RRMSPSV_0000_9965_20190110_041a3aae-01f4-406d-bbce-74a9b1118d39.xml
VO_RRMSPSV_0000_9965_20190110_0462ae43-4fb5-49ce-aa4d-93138b1b2afa.xml
VO_RRMSPSV_0000_9965_20190110_0469f304-8a4d-49e7-a0d3-cc106230fdc0.xml
VO_RRMSPSV_0000_9965_20190110_046db543-e3cb-41d0-a08d-d233302aa94b.xml
VO_RRMSPSV_0000_9965_20190110_0472bfb7-9bd1-4e94-9957-ae3bd44658b7.xml
VO_RRMSPSV_0000_9965_20190110_0480ddc1-0c14-4b56-9b94-e7a0f67f22d5.xml
VO_RRMSPSV_0000_9965_20190110_04823423-45b9-466d-9533-85e29d5761c6.xml
VO_RRMSPSV_0000_9965_20190110_0482372b-226a-47b1-98ef-ccd96aeecc2a.xml
VO_RRMSPSV_0000_9965_20190110_0487c436-b910-40cc-b1e7-29492ebe0814.xml
VO_RRMSPSV_0000_9965_20190110_048d792b-89fb-4039-8e50-95a7b87df574.xml
VO_RRMSPSV_0000_9965_20190110_04a48259-57b7-4ed9-a001-f55144021695.xml
VO_RRMSPSV_0000_9965_20190110_04a703e1-d63f-4217-b28e-8545e7496cd4.xml
VO_RRM

VO_RRMSPSV_0000_9965_20190110_097f2cfd-ae4e-43f2-af8b-69fd57ff7c23.xml
VO_RRMSPSV_0000_9965_20190110_099698c1-6117-4c21-b1d9-13ba9b7c57e6.xml
VO_RRMSPSV_0000_9965_20190110_09982283-b293-45fe-9562-9ec533dc86d9.xml
VO_RRMSPSV_0000_9965_20190110_0999c6f6-c223-4278-a0ac-158db5f64683.xml
VO_RRMSPSV_0000_9965_20190110_099fbb15-c528-423c-841b-af3d14b346d0.xml
VO_RRMSPSV_0000_9965_20190110_09aa3eab-63af-467d-931c-6fd631f691f5.xml
VO_RRMSPSV_0000_9965_20190110_09ab116b-0985-426a-a780-eab0b8ae5cce.xml
VO_RRMSPSV_0000_9965_20190110_09ae7780-cbbd-462c-b25a-c1279d61087d.xml
VO_RRMSPSV_0000_9965_20190110_05cdd2ef-960a-47d0-982f-ab7628facb47.xml
VO_RRMSPSV_0000_9965_20190110_0677e574-8476-4497-a59e-ec23b34f3ce7.xml
VO_RRMSPSV_0000_9965_20190110_071174bd-966d-45ba-8298-1076d5a6f652.xml
VO_RRMSPSV_0000_9965_20190110_07648829-b247-46e5-8afe-ecde7f7aa01b.xml
VO_RRMSPSV_0000_9965_20190110_07be5f0f-37ac-49e7-86ec-29a570e46196.xml
VO_RRMSPSV_0000_9965_20190110_0846e014-3c37-437c-a726-04c408d77964.xml
VO_RRM

VO_RRMSPSV_0000_9965_20190110_0e0d7832-04eb-4d77-8822-ad08162ab64b.xml
VO_RRMSPSV_0000_9965_20190110_0e2f394d-0e01-4536-907b-ef93627a208b.xml
VO_RRMSPSV_0000_9965_20190110_0e30c85f-d0f0-4067-bc00-eb35b8c082ce.xml
VO_RRMSPSV_0000_9965_20190110_0e3427b9-3841-4e26-9395-09f6b5c0f67d.xml
VO_RRMSPSV_0000_9965_20190110_0e38f7e2-4616-4c66-be87-3fa80aa693ab.xml
VO_RRMSPSV_0000_9965_20190110_0e41ebef-8554-4072-b787-bdc8fef7f5c2.xml
VO_RRMSPSV_0000_9965_20190110_0e4d3b5f-539e-4964-ae4f-a1a27f95222b.xml
VO_RRMSPSV_0000_9965_20190110_0e54f866-51f5-4522-917a-866518d1989a.xml
VO_RRMSPSV_0000_9965_20190110_0e6eadf6-3bdb-484a-94c3-734c9492abb1.xml
VO_RRMSPSV_0000_9965_20190110_0e76790d-c440-4d59-aee1-43a5a410408a.xml
VO_RRMSPSV_0000_9965_20190110_0e839b6e-00b4-4ed0-b06e-5e2c8cf422f8.xml
VO_RRMSPSV_0000_9965_20190110_0e92a78e-2e45-47db-a956-ba976d4faf36.xml
VO_RRMSPSV_0000_9965_20190110_0eb51b57-39a2-4234-b047-f07c20cef3c8.xml
VO_RRMSPSV_0000_9965_20190110_0ec51f2f-4e74-4328-9ad7-2c399d8b517c.xml
VO_RRM

VO_RRMSPSV_0000_9965_20190110_12bf471f-6e38-4851-b64a-b90c8081fe51.xml
VO_RRMSPSV_0000_9965_20190110_12c882fa-5b9b-4829-8435-076390c4c450.xml
VO_RRMSPSV_0000_9965_20190110_12d61358-5a6d-4106-8a83-c070ea7c58ee.xml
VO_RRMSPSV_0000_9965_20190110_12d8817d-0634-427b-90af-a540a4080656.xml
VO_RRMSPSV_0000_9965_20190110_12de4e0f-c194-4cdf-a717-1354a30eacca.xml
VO_RRMSPSV_0000_9965_20190110_12e22060-43a3-4bd0-93bf-194f598cafd4.xml
VO_RRMSPSV_0000_9965_20190110_12f918ef-dd23-4033-80f5-015f3821e56a.xml
VO_RRMSPSV_0000_9965_20190110_12fbe787-8c45-4b50-ae32-52c09039897f.xml
VO_RRMSPSV_0000_9965_20190110_1309769c-0721-4db9-92bd-07139667b003.xml
VO_RRMSPSV_0000_9965_20190110_1310da2a-ed34-4a81-bd21-d2aea3c316e7.xml
VO_RRMSPSV_0000_9965_20190110_131ca5b9-7633-459b-a17c-eee83a0dff16.xml
VO_RRMSPSV_0000_9965_20190110_132e02b3-882c-4f57-bd88-087f3699e2b6.xml
VO_RRMSPSV_0000_9965_20190110_13341fc1-dfd3-4216-abe2-85a53d94de8f.xml
VO_RRMSPSV_0000_9965_20190110_133422a7-6626-4356-a2eb-14507d01b0b1.xml
VO_RRM

VO_RRMSPSV_0000_9965_20190110_168863a6-dadd-4415-8365-abcf0f36a900.xml
VO_RRMSPSV_0000_9965_20190110_168a1552-08ef-4b01-878e-8919ef98cd08.xml
VO_RRMSPSV_0000_9965_20190110_1693ca21-915e-47a1-abb8-41777aaeafa6.xml
VO_RRMSPSV_0000_9965_20190110_16a2385a-95b9-4539-8f50-6b0135ac8e3f.xml
VO_RRMSPSV_0000_9965_20190110_16a7745c-963c-4d02-8869-15a529fd24b8.xml
VO_RRMSPSV_0000_9965_20190110_16c5144f-dbb8-48cf-995a-dfc586d89ba8.xml
VO_RRMSPSV_0000_9965_20190110_16d61720-afe3-49e4-96ed-94e403e98146.xml
VO_RRMSPSV_0000_9965_20190110_16dd90c1-7a30-4a18-a4a2-2aa8e3f9e0fe.xml
VO_RRMSPSV_0000_9965_20190110_16e64cb5-d902-4349-98e8-622573e42f04.xml
VO_RRMSPSV_0000_9965_20190110_16ecb140-d7bd-4bd2-b61a-559de69cddb2.xml
VO_RRMSPSV_0000_9965_20190110_16f95bed-3a5b-4869-aafa-2583a634a848.xml
VO_RRMSPSV_0000_9965_20190110_171105e3-8879-46d7-a8bd-69d4be4ef9ab.xml
VO_RRMSPSV_0000_9965_20190110_17138ae7-e56e-44fe-bace-3374844f170e.xml
VO_RRMSPSV_0000_9965_20190110_1720a8d0-6fb2-4916-aa37-28d0dd5648c1.xml
VO_RRM

463.1250970363617